In [1]:
import sys
from pathlib import Path

# add parent folder to the path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

___

In [2]:
import numpy as np
import torch
import torch.nn as nn
from src.models.base import SimpleNet 
from src.dataloader import DatasetONT
from torch.utils.data import DataLoader

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
dataset_train = DatasetONT(recfile="../data/subsample_train.hdf5")
dataset_val   = DatasetONT(recfile="../data/subsample_val.hdf5")

In [5]:
loader = iter(DataLoader(dataset_val, batch_size=2, shuffle=True))

In [6]:
x,y = next(loader)
x.shape, y.shape

(torch.Size([2, 1, 4096]), torch.Size([2, 271]))

In [8]:
model = SimpleNet(n_channels = 1, n_classes = 271)
model.eval()

SimpleNet(
  (conv1): Conv1d(1, 20, kernel_size=(5,), stride=(1,))
  (relu1): ReLU()
  (maxpool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(20, 50, kernel_size=(5,), stride=(1,))
  (relu2): ReLU()
  (maxpool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten1): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=51050, out_features=500, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=500, out_features=271, bias=True)
)

In [9]:
output = model(x)
output.shape

torch.Size([2, 271])